In [2]:
import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

2022-12-05 14:26:17.961324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 14:26:18.074696: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 14:26:18.074713: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-05 14:26:18.093157: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-05 14:26:18.553194: W tensorflow/stream_executor/platform/de

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import math as mt
import random as rd
import matplotlib.pyplot as plt
import scipy.stats
from numpy.lib.arraysetops import intersect1d

# ignore some of above, trying a different model!
from sklearn.datasets import load_linnerud
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

# for classifier 
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# other
from utils import type_to_int, type_to_int_dict_inv

In [4]:
poke = pd.read_csv("pokemon_clean.csv") #import Pokemon data
training_data = pd.read_csv("trainingdata2202.csv") #import training data

poke = poke[poke.is_legendary == 0] #clean data for no legendaries
poke.reset_index(drop = True,inplace = True)
poke = poke.drop([131, 195, 196, 229, 348, 717]) #clean out problem children (Ditto, etc)
poke.reset_index(drop = True,inplace = True)

In [5]:
display(training_data.head())
X = training_data.iloc[:,1:12]
Y = training_data.iloc[:,12:-1 ]

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,0,Trevenant,Grimer,Sawk,Zorua,Tangela,Meowstic,Vulpix,Forretress,Wishiwashi,...,Hydreigon,Hypno,Delphox,Bisharp,Blastoise,Empoleon,Jolteon,Oranguru,Slowking,Kangaskhan
1,1,Arcanine,Tyrantrum,Venusaur,Budew,Kangaskhan,Braixen,Lickilicky,Victreebel,Espeon,...,Sneasel,Weavile,Greninja,Scizor,Slaking,Minior,Goodra,Ribombee,Lucario,Aerodactyl
2,2,Sawk,Butterfree,Poliwhirl,Tynamo,Seaking,Floette,Quagsire,Litten,Cloyster,...,Silvally,Seviper,Alakazam,Swalot,Oricorio,Delphox,Nidoqueen,Salazzle,Steelix,Typhlosion
3,3,Butterfree,Wurmple,Kangaskhan,Vibrava,Corsola,Toucannon,Hitmonlee,Ursaring,Karrablast,...,Greninja,Alakazam,Manectric,Jellicent,Absol,Mienshao,Porygon-Z,Arcanine,Aegislash,Chimecho
4,4,Elgyem,Mamoswine,Ambipom,Greninja,Caterpie,Toucannon,Magcargo,Stunky,Torracat,...,Purrloin,Liepard,Pansage,Simisage,Pansear,Simisear,Servine,Phione,Froslass,Gallade


In [6]:
# import precomputed feature data
pokemon_features_df = pd.read_csv('pokemon_features.csv', index_col='name') 
# separate feature data into types and stats
type_cols = pokemon_features_df.columns[:18]
stat_cols = pokemon_features_df.columns[18:]
# construct feature vectors for training data 
typesX = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in X.iterrows()], axis=0)
statsX = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in X.iterrows()], axis=0)
typesY = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in Y[['11']].iterrows()], axis=0)
statsY = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in Y[['11']].iterrows()], axis=0)

print(typesX[0], statsX[0], '\n')
print(typesY[0], statsY[0])

[0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0] [474 166 167 325 105 130 465 210 150 330 145  80 435 160 180 466 187 155
 299 115 103 465 130 215 620 170 180 535 220 165 355 131 122] 

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0] [600 325 160]


In [7]:
def gotta_match_it_all(types_to_be_matched, stats_to_be_matched):
    top_num = 5
    types_matched = list(pokemon_features_df.index[((pokemon_features_df[type_cols] == 1) & (pokemon_features_df[type_cols] == types_to_be_matched)).any(axis=1)])
    pkm_matched_all = (pokemon_features_df[stat_cols] - stats_to_be_matched).abs().sum(axis=1).sort_values()
    if (len(types_matched) != 0):
        pkm_matched_with_type = pkm_matched_all[types_matched].sort_values()
        return pkm_matched_with_type.head(top_num)
    else:
        return pkm_matched_all.head(top_num)

In [8]:
def compare_results(ground_truth, pred_types, pred_stats, start_compare, end_compare, test_index):
    test_size = len(test_index)
    pred = [gotta_match_it_all(pred_types[i], pred_stats[i]).index for i in range(test_size)]
    truth_to_compare = ground_truth.iloc[:,start_compare:end_compare]
    accurate = [intersect1d(pred[i], truth_to_compare.iloc[test_index[i]]) for i in range(test_size)]
    return np.average([len(accurate[i]) for i in range(test_size)])

In [9]:
# if we were to run the model once

# randomly select data points to set aside for testing
data_total = len(Y)
test_size = mt.floor(data_total/6)
test_index = sorted(rd.sample(range(0, data_total), test_size))
training_index = list(set(range(data_total)) - set(test_index))
typesX_test = typesX[test_index]
statsX_test = statsX[test_index]
typesX_train = typesX[training_index]
statsX_train = statsX[training_index]
typesY_train = typesY[training_index]
statsY_train = statsY[training_index]

# training
clf = MultiOutputClassifier(LogisticRegression()).fit(typesX_train, typesY_train)
regr = MultiOutputRegressor(Ridge(random_state = 123)).fit(statsX_train, statsY_train)
# predicting on test data
typesPredict = clf.predict(typesX_test)
statsPredict=regr.predict(statsX_test)

In [10]:
# now we want to compare multiple runs
num_val = 10
performance = np.zeros(num_val)

for i in range(num_val):
    data_total = len(Y)
    test_size = mt.floor(data_total/6)
    test_index = sorted(rd.sample(range(0, data_total), test_size))
    training_index = list(set(range(data_total)) - set(test_index))

    typesX_test = typesX[test_index]
    statsX_test = statsX[test_index]
    typesX_train = typesX[training_index]
    statsX_train = statsX[training_index]
    typesY_train = typesY[training_index]
    statsY_train = statsY[training_index]

    clf = MultiOutputClassifier(LogisticRegression()).fit(typesX_train, typesY_train)
    typesPredict = clf.predict(typesX_test)
    regr = MultiOutputRegressor(Ridge(random_state = 123)).fit(statsX_train, statsY_train)
    statsPredict=regr.predict(statsX_test)

    performance[i] =  compare_results(Y, typesPredict, statsPredict, 0, 10, test_index)

In [11]:
print(performance, '\n')
print("average intersection: ", np.mean(performance))

[1.01362398 0.95640327 0.90190736 0.95095368 0.96730245 0.92643052
 0.96730245 0.90190736 0.97275204 0.91280654] 

average intersection:  0.9471389645776567


## **CHAMPIONSHIP TEAMS WITH USER INPUT**

In [12]:
# Championship Teams
championship_teams = np.array([["Milotic","Gothitelle","Volcarona","Escavalier","Conkeldurr","Hydreigon"],
                      ["Milotic","Aerodactyl","Scrafty","Eelektross","Krookodile","Haxorus"],
                      ["Milotic","Cofagrigus","Scrafty","Reuniclus","Conkeldurr","Emboar"],
                      ["Salamence","Electivire","Incineroar","Gastrodon","Snorlax","Ferrothorn"],
                      ["Gengar","Sceptile","Gliscor","Incineroar","Haxorus","Politoed"],
                      ["Metagross","Electivire","Gliscor","Incineroar","Mimikyu","Snorlax"],
                      ["Gyarados","Garchomp","Talonflame","Pachirisu","Gothitelle","Gardevoir"],
                      ["Charizard","Lucario","Garchomp","Mamoswine","Tyranitar","Salamence"],
                      ["Mawile","Conkeldurr","Eelektross","Hydreigon","Tyranitar","Aegislash"],
                      ["Altaria","Gardevoir","Marowak","Krookodile","Whimsicott","Skarmory"],
                      ["Espeon", "Arcanine", "Garchomp", "Ninetales", "Mandibuzz", "Ampharos"],
                      ["Kingdra","Gyarados","Marowak","Garchomp","Oranguru","Ferrothorn"]])
user_input = np.array(["Gyarados","Raichu","Quagsire","Cloyster","Slowking"])
user_input = np.reshape(np.tile(user_input,[len(championship_teams)]),[len(championship_teams),len(user_input)])
combined  = np.concatenate((user_input,championship_teams),axis = 1)
pdCombined = pd.DataFrame(combined,columns =[0,1,2,3,4,5,6,7,8,9,10])
pdCombined

,0,1,2,3,4,5,6,7,8,9,10
0,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Milotic,Gothitelle,Volcarona,Escavalier,Conkeldurr,Hydreigon
1,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Milotic,Aerodactyl,Scrafty,Eelektross,Krookodile,Haxorus
2,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Milotic,Cofagrigus,Scrafty,Reuniclus,Conkeldurr,Emboar
3,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Salamence,Electivire,Incineroar,Gastrodon,Snorlax,Ferrothorn
4,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Gengar,Sceptile,Gliscor,Incineroar,Haxorus,Politoed
5,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Metagross,Electivire,Gliscor,Incineroar,Mimikyu,Snorlax
6,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Gyarados,Garchomp,Talonflame,Pachirisu,Gothitelle,Gardevoir
7,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Charizard,Lucario,Garchomp,Mamoswine,Tyranitar,Salamence
8,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Mawile,Conkeldurr,Eelektross,Hydreigon,Tyranitar,Aegislash
9,Gyarados,Raichu,Quagsire,Cloyster,Slowking,Altaria,Gardevoir,Marowak,Krookodile,Whimsicott,Skarmory


In [13]:
typesXuser = np.stack([pokemon_features_df.loc[row,type_cols].to_numpy().reshape(-1) for _,row in pdCombined.iterrows()], axis=0)
statsXuser = np.stack([pokemon_features_df.loc[row,stat_cols].to_numpy().reshape(-1) for _,row in pdCombined.iterrows()], axis=0)
typesPredictUser = clf.predict(typesXuser)
statsPredictUser = regr.predict(statsXuser)

best_mons = np.empty(60, dtype="<U12")
for i in range(12):
  for j in range(5):
    best_mons[i+j]=gotta_match_it_all(typesPredictUser[i],statsPredictUser[i]).index[j]
best_mons

array(['Greninja', 'Salamence', 'Charizard', 'Charizard', 'Salamence',
       'Tyranitar', 'Salamence', 'Salamence', 'Salamence', 'Charizard',
       'Greninja', 'Slaking', 'Garchomp', 'Salamence', 'Metagross',
       'Charizard', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
      dtype='<U12')

In [14]:
gotta_match_it_all(typesPredictUser[1],statsPredictUser[1]).index[0]
best_mons = np.empty(60, dtype="<U12")
best_mons[1] = "Salamence"

best_mons

array(['', 'Salamence', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', '', '', '', '', '', '', '', '', ''], dtype='<U12')